In [179]:
import pandas as pd

df = pd.read_csv("habitats.csv", index_col="Unnamed: 0")
subs = [("♀", "-F"), ("♂", "-M"), ("'", "’"), ("Flabébé", "Flabébé"), ("Daschsbun", "Dachsbun")]
for old, new in subs:
    df["species"] = df.species.str.replace(old, new)
df

,species,habitat
0,Bulbasaur,Kantonian forests and jungles
1,Ivysaur,Kantonian forests and jungles
2,Venusaur,Kantonian forests and jungles
3,Charmander,Kantonian mountains and volcanoes
4,Charmeleon,Kantonian mountains and volcanoes
...,...,...
1003,Chi-Yu,Ruins
1004,Roaring Moon,Area Zero
1005,Iron Valiant,Area Zero
1006,Koraidon,Anywhere


In [180]:
dex = pd.read_csv("../dex/dex.csv")
dex = dex[dex.num > 0]
dex = dex[~dex.forme.isin(["Mega", "Mega-X", "Mega-Y", "Primal", "Gmax"])]
dex = dex[["num", "name", "baseSpecies", "prevo", "legendary"]]
dex.baseSpecies.fillna(value=dex.name, inplace=True)
dex = dex.merge(right=dex[["name", "legendary"]], how="left", left_on="baseSpecies", right_on="name", suffixes=["_1", None])
dex.drop(columns=["name_1", "legendary_1"], inplace=True)
dex

,num,baseSpecies,prevo,name,legendary
0,1,Bulbasaur,NaN,Bulbasaur,NaN
1,2,Ivysaur,Bulbasaur,Ivysaur,NaN
2,3,Venusaur,Ivysaur,Venusaur,NaN
3,4,Charmander,NaN,Charmander,NaN
4,5,Charmeleon,Charmander,Charmeleon,NaN
...,...,...,...,...,...
1213,1006,Iron Valiant,NaN,Iron Valiant,Paradox
1214,1007,Koraidon,NaN,Koraidon,Restricted Legendary
1215,1008,Miraidon,NaN,Miraidon,Restricted Legendary
1216,1009,Walking Wake,NaN,Walking Wake,Paradox


In [181]:
dex.columns

Index(['num', 'baseSpecies', 'prevo', 'name', 'legendary'], dtype='object')

In [182]:
dex = dex.merge(right=df, how="left", left_on="baseSpecies", right_on="species")
dex

,num,baseSpecies,prevo,name,legendary,species,habitat
0,1,Bulbasaur,NaN,Bulbasaur,NaN,Bulbasaur,Kantonian forests and jungles
1,2,Ivysaur,Bulbasaur,Ivysaur,NaN,Ivysaur,Kantonian forests and jungles
2,3,Venusaur,Ivysaur,Venusaur,NaN,Venusaur,Kantonian forests and jungles
3,4,Charmander,NaN,Charmander,NaN,Charmander,Kantonian mountains and volcanoes
4,5,Charmeleon,Charmander,Charmeleon,NaN,Charmeleon,Kantonian mountains and volcanoes
...,...,...,...,...,...,...,...
1213,1006,Iron Valiant,NaN,Iron Valiant,Paradox,Iron Valiant,Area Zero
1214,1007,Koraidon,NaN,Koraidon,Restricted Legendary,Koraidon,Anywhere
1215,1008,Miraidon,NaN,Miraidon,Restricted Legendary,Miraidon,Anywhere
1216,1009,Walking Wake,NaN,Walking Wake,Paradox,NaN,NaN


In [183]:
import re

habitat_map = {
    "grassland": "field|grassland|plain|meadow|scrubland",
    "forest": "forest|jungle|woodland",
    "waters-edge": "beach|lake|estuarie|pond|riverside|swamp|tropical river|wetland",
    "sea": "coast|ocean|sea",
    "cave": "cave",
    "mountain": "glacier|mountain|peak|tundra|volcano",
    "rough-terrain": "badland|wasteland|desert|rocky|ruin",
    "urban": "building|computer|human|sewer|power plant|urban",
    "rare": "unknown"
}

for habitat, keywords in habitat_map.items():
    dex[habitat] = dex.habitat.str.contains(keywords, flags=re.IGNORECASE, regex=True)

dex.loc[dex.prevo == "Eevee", "urban"] = True
dex.loc[~dex.legendary.isna(), "rare"] = True
dex.loc[dex.baseSpecies == "Unown", "rare"] = True
dex.loc[dex.baseSpecies == "Castform", "grassland"] = True

dex["habitat_count"] = dex[list(habitat_map)].sum(axis=1)
dex


,num,baseSpecies,prevo,name,legendary,species,habitat,grassland,forest,waters-edge,sea,cave,mountain,rough-terrain,urban,rare,habitat_count
0,1,Bulbasaur,NaN,Bulbasaur,NaN,Bulbasaur,Kantonian forests and jungles,False,True,False,False,False,False,False,False,False,1.0
1,2,Ivysaur,Bulbasaur,Ivysaur,NaN,Ivysaur,Kantonian forests and jungles,False,True,False,False,False,False,False,False,False,1.0
2,3,Venusaur,Ivysaur,Venusaur,NaN,Venusaur,Kantonian forests and jungles,False,True,False,False,False,False,False,False,False,1.0
3,4,Charmander,NaN,Charmander,NaN,Charmander,Kantonian mountains and volcanoes,False,False,False,False,False,True,False,False,False,1.0
4,5,Charmeleon,Charmander,Charmeleon,NaN,Charmeleon,Kantonian mountains and volcanoes,False,False,False,False,False,True,False,False,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213,1006,Iron Valiant,NaN,Iron Valiant,Paradox,Iron Valiant,Area Zero,False,False,False,False,False,False,False,False,True,1.0
1214,1007,Koraidon,NaN,Koraidon,Restricted Legendary,Koraidon,Anywhere,False,False,False,False,False,False,False,False,True,1.0
1215,1008,Miraidon,NaN,Miraidon,Restricted Legendary,Miraidon,Anywhere,False,False,False,False,False,False,False,False,True,1.0
1216,1009,Walking Wake,NaN,Walking Wake,Paradox,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1.0


In [186]:
dex[dex.habitat_count > 1]

,num,baseSpecies,prevo,name,legendary,species,habitat,grassland,forest,waters-edge,sea,cave,mountain,rough-terrain,urban,rare,habitat_count
15,16,Pidgey,NaN,Pidgey,NaN,Pidgey,"Kanto and Johtonian forests, fields, and grass...",True,True,False,False,False,False,False,False,False,2.0
16,17,Pidgeotto,Pidgey,Pidgeotto,NaN,Pidgeotto,Kantonian and Johto forests and fields,True,True,False,False,False,False,False,False,False,2.0
17,18,Pidgeot,Pidgeotto,Pidgeot,NaN,Pidgeot,Kantonian and Johtonian forests and fields,True,True,False,False,False,False,False,False,False,2.0
18,19,Rattata,NaN,Rattata,NaN,Rattata,"Fields, plains, savannas, urban areas",True,False,False,False,False,False,False,True,False,2.0
19,19,Rattata,NaN,Rattata,NaN,Rattata,"Fields, plains, savannas, urban areas",True,False,False,False,False,False,False,True,False,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204,998,Baxcalibur,Arctibax,Baxcalibur,NaN,Baxcalibur,"Snow-packed caves, boreal forests",False,True,False,False,True,False,False,False,False,2.0
1208,1001,Wo-Chien,NaN,Wo-Chien,Sub-Legendary,Wo-Chien,Ruins,False,False,False,False,False,False,True,False,True,2.0
1209,1002,Chien-Pao,NaN,Chien-Pao,Sub-Legendary,Chien-Pao,Ruins,False,False,False,False,False,False,True,False,True,2.0
1210,1003,Ting-Lu,NaN,Ting-Lu,Sub-Legendary,Ting-Lu,Ruins,False,False,False,False,False,False,True,False,True,2.0
